In [7]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d 
import numpy
import sympy
from sympy.solvers import solve
from sympy import diff
from time import sleep
from progressbar import progressbar

MAX_ITERATION_QUASE_NEWTON = 100
MAX_ITERATIONITERATION_DICOTOMICA = 1000

In [8]:
x, y, a = sympy.symbols("x y a")
#função 1
f1 = 10*pow(y-x*x, 2)+pow(1-x, 2)

#função 2
f2 = 200*pow(y-x*x, 2)+pow(1-x, 2)

#função 3
A3 = [[ 0.78, -0.02, -0.12, -0.14],
     [-0.02,  0.86, -0.04,  0.06],
     [-0.12, -0.04,  0.72, -0.08],
     [-0.14,  0.06, -0.08, 0.74]]
B3 = [0.76, 0.08, 1.12, 0.68]
soma_temp_a3 = sum([sum([A3[m][n] for n in range(len(A3))]) for m in range(len(A3))])
f3 = 0.5*x*x*soma_temp_a3 - x*sum([i for i in B3])

#função 4
A4 = [[1, 1, 1, 1],
     [1, 3, 3, 3],
     [1, 3, 5, 5],
     [1, 3, 5, 7]]
B4 = [10, 28, 42, 50]
soma_temp_a4 = sum([sum([A4[m][n] for n in range(len(A4))]) for m in range(len(A4))])
f4 = 0.5*x*x*soma_temp_a4 - x*sum([i for i in B4])

In [9]:
def busca_dicotomica(a_min, a_max, e, f_alpha):
    for k in range(MAX_ITERATIONITERATION_DICOTOMICA):
        if abs(a_max/a_min) < 1 + e*10 and k > 5:
                break
        lambda_k = ((a_min+a_max)/2)-e
        mi_k = ((a_min+a_max)/2)+e
        f_lambda = f_alpha.subs(a, lambda_k)
        f_mi = f_alpha.subs(a, mi_k)
            
        if f_lambda < f_mi:
            a_max = mi_k
        else:
            a_min = lambda_k
    return (a_min+a_max)/2

In [10]:
def DFP(input_D_i, input_s_i, input_y_i):
    s_i = numpy.array([[i] for i in input_s_i])
    y_i = numpy.array([[i] for i in input_y_i])
    D_i = numpy.array(input_D_i)
    p1 = numpy.true_divide(s_i.dot(s_i.transpose()), s_i.transpose().dot(y_i))
    div_p2 = y_i.transpose().dot(D_i).dot(y_i)
    nom_p2 = D_i.dot(y_i).dot(y_i.transpose()).dot(D_i)
    p2 = numpy.true_divide(nom_p2, div_p2)
    return D_i + p1 - p2

In [14]:
def metodo_quase_newton(x_i, e, f):
    grad = [f.diff(x), f.diff(y)]
    
    #Passo 3
    D_i = numpy.identity(len(x_i))
    
    for i in progressbar(range(MAX_ITERATION_QUASE_NEWTON)):
        #Passo 1
        gradiente = [grad[0].subs([(x, x_i[0]), (y, x_i[1])]), 
                     grad[1].subs([(x, x_i[0]), (y, x_i[1])])]
        #print(f"gradiente = {gradiente}")

        #Passo 4
        d_i = -D_i.dot(gradiente)
        #print(f"d_i = {d_i}")
        
        #Passo 5
        f_alpha = f.subs([(x, x_i[0] + d_i[0] * a), 
                          (y, x_i[1] + d_i[1] * a)])
        alpha = busca_dicotomica(-10, 10, e/10, f_alpha)
        #print(f"alpha = {alpha}")

        #Passo 6
        antigo_x_i = x_i
        x_i = [x_i[k] + alpha*d_i[k] for k in range(len(x_i))]
        #print(f"x_i = {x_i}")
        
        antigo_gradiente = gradiente
        gradiente = [grad[0].subs([(x, x_i[0]), (y, x_i[1])]), 
                     grad[1].subs([(x, x_i[0]), (y, x_i[1])])]
        
        #Passo 7
        convergiu = True
        for k in range(len(x_i)):
            if abs(antigo_x_i[k] - x_i[k]) > e/1000:
                convergiu = False
                break
        if convergiu:
            print(f"Numero de iterações foi o suficiente para convergir")
            break
            
        #Passo 8
        s_i = [x_i[k] - antigo_x_i[k] for k in range(len(x_i))]
        y_i = [gradiente[k] - antigo_gradiente[k] for k in range(len(gradiente))]
        #print(f"s_i = {s_i}")
        #print(f"y_i = {y_i}")
        
        #Passo 9
        np_s_i = numpy.array(s_i)
        np_y_i = numpy.array(y_i)
        result = numpy.dot(np_s_i.transpose(), np_y_i)
        
        #Passo 9
        if result < 0:
            D_i = numpy.identity(len(x_i))
        
        #Passo 10
        if result >= 0:
            D_i = DFP(D_i, np_s_i, np_y_i)
        #print(f"D_i+1 = {D_i}")
        #print(f"result = {result}")
        
    return x_i

In [15]:
mqn1 = metodo_quase_newton([-1.2, 1], 0.001, f1)
sp1 = solve([f1.diff(x), f1.diff(y)], [x, y])
print(f"\nFunção 1 - Valor cálculado pelo método Quase-Newton : {mqn1}\nValor otimizado : {sp1}")

  9% (9 of 100) |##                      | Elapsed Time: 0:00:01 ETA:   0:00:15

Numero de iterações foi o suficiente para convergir

Função 1 - Valor cálculado pelo método Quase-Newton : [1.00000000000021, 1.00000000000074]
Valor otimizado : [(1, 1)]


In [22]:
mqn2 = metodo_quase_newton([-1.2, 1], 0.001, f2)
sp2 = solve([f2.diff(x), f2.diff(y)], [x, y])
print(f"\nFunção 2 - Valor cálculado pelo método Quase-Newton : {mqn2}\nValor otimizado : {sp2}")

 15% (15 of 100) |###                    | Elapsed Time: 0:00:02 ETA:   0:00:13

Numero de iterações foi o suficiente para convergir

Função 2 - Valor cálculado pelo método Quase-Newton : [0.999999999999647, 0.999999999999303]
Valor otimizado : [(1, 1)]


In [21]:
mqn3 = metodo_quase_newton([0, 0], 0.001, f3)
sp3 = solve(f3.diff(x), x)
print(f"\nFunção 3 - Valor cálculado pelo método Quase-Newton : {mqn3[0]}\nValor otimizado : {sp3[0]}")

  2% (2 of 100) |                        | Elapsed Time: 0:00:00 ETA:  00:00:00

Numero de iterações foi o suficiente para convergir

Função 3 - Valor cálculado pelo método Quase-Newton : 1.09090911388615
Valor otimizado : 1.09090909090909


In [18]:
mqn4 = metodo_quase_newton([0, 0], 0.001, f4)
sp4 = solve(f4.diff(x), x)
print(f"\nFunção 4 - Valor cálculado pelo método Quase-Newton : {mqn4[0]}\nValor otimizado : {sp4[0]}\n\n\n")

  1% (1 of 100) |                        | Elapsed Time: 0:00:00 ETA:   0:00:20

Numero de iterações foi o suficiente para convergir

Função 4 - Valor cálculado pelo método Quase-Newton : 2.95454545454574
Valor otimizado : 2.95454545454545



